In [ ]:
import requests
import json
from IPython.display import JSON
from keys import *

In [2]:
#be careful not to run too often as there is a maximum of 200 api calls per month

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/SLLP/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [3]:
if response.status_code == 200:
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [5]:
flight_arrivals = response.json()

for flight in flight_arrivals['arrivals']:
    scheduled_time = flight['arrival'].get('scheduledTime')
    number = flight.get('number')
    terminal = flight['arrival'].get('terminal')
    departure_airport = flight['departure']['airport'].get('name')
    aircraft_model = flight['aircraft'].get('model')
    airline_name = flight['airline'].get('name')
    # You can print or process these variables as needed

In [6]:
import pandas as pd
from numpy import nan

flight_arrivals = response.json()

def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
        'aircraft' : flights['aircraft']['model']
    })
    return(flights_df)

flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

flight_arrivals.head()

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,OB 602,Cochabamba,BoA,Boeing 737,2025-09-01 12:05Z,2025-09-01 08:05-04:00
0,OB 663,Santa Cruz de la Sierra,BoA,Boeing 737-800,2025-09-01 12:40Z,2025-09-01 08:40-04:00
0,OB 561,Sucre,BoA,Bombardier CRJ,2025-09-01 13:40Z,2025-09-01 09:40-04:00
0,OB 305,Quijarro,BoA,Boeing 737-800,2025-09-01 14:15Z,2025-09-01 10:15-04:00
0,OB 606,Cochabamba,BoA,Boeing 737,2025-09-01 14:55Z,2025-09-01 10:55-04:00
